In [1]:
#!pip install emmv
#!pip install ipympl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
import matplotlib
import matplotlib.dates as mdates
%matplotlib inline
%matplotlib widget
warnings.filterwarnings("ignore")
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
import matplotlib.pyplot as plt
from matplotlib import pyplot
#import plotly.graph_objs as go
#init_notebook_mode(connected=True)
import eif as iso
from functions import load_dataset,calc_percent_NAs, adfuller_test,data_load
from sklearn import preprocessing
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, calinski_harabasz_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from emmv import emmv_scores
from sklearn.metrics import silhouette_score


In [2]:
data = load_dataset("anomaly1.csv")
data

name,Flow,P_1,P_3,P_4,P_5,T_2,T_3,T_4
Datetime,,,,,,,,
2021-03-02 10:10:00,48748.772049,2.068251,2.184192,1.416815,NaN,21.069200,21.038382,20.848113
2021-03-02 10:10:01,48781.154251,2.067713,2.183735,NaN,1.054952,21.069885,NaN,20.854292
2021-03-02 10:10:02,48803.442764,2.067444,NaN,1.416587,NaN,21.075372,21.047293,NaN
2021-03-02 10:10:03,48660.002518,2.068251,2.184649,1.417272,NaN,21.080173,21.052776,20.852919
2021-03-02 10:10:04,48702.492142,2.066905,2.184192,1.416815,NaN,21.087717,21.057574,20.853606
...,...,...,...,...,...,...,...,...
2021-03-02 15:43:55,47924.371719,1.893587,2.056611,1.391213,1.048328,28.196283,28.248115,28.052040
2021-03-02 15:43:56,47945.698929,1.894125,2.057068,1.390984,1.049013,28.190796,28.249486,28.042429
2021-03-02 15:43:57,47874.507523,1.893587,2.056840,NaN,1.048100,28.199026,NaN,28.041742


In [3]:
#### Lets see missing values 
missing = calc_percent_NAs(data)
cols = data.columns
index = pd.to_datetime(data.index,utc = True)
##Missing values imputation
lr = LinearRegression()
imp = IterativeImputer(estimator=lr,missing_values=np.nan, max_iter=10, verbose=2, imputation_order='roman',random_state=0)
data = imp.fit_transform(data)
data = pd.DataFrame(data, columns=cols, index = index)
data

[IterativeImputer] Completing matrix with shape (19918, 8)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.06
[IterativeImputer] Change: 11.607349258616095, scaled tolerance: 49.0321678161621 
[IterativeImputer] Early stopping criterion reached.


name,Flow,P_1,P_3,P_4,P_5,T_2,T_3,T_4
Datetime,,,,,,,,
2021-03-02 10:10:00+00:00,48748.772049,2.068251,2.184192,1.416815,1.055568,21.069200,21.038382,20.848113
2021-03-02 10:10:01+00:00,48781.154251,2.067713,2.183735,1.417930,1.054952,21.069885,21.007326,20.854292
2021-03-02 10:10:02+00:00,48803.442764,2.067444,2.185515,1.416587,1.055131,21.075372,21.047293,20.865555
2021-03-02 10:10:03+00:00,48660.002518,2.068251,2.184649,1.417272,1.055773,21.080173,21.052776,20.852919
2021-03-02 10:10:04+00:00,48702.492142,2.066905,2.184192,1.416815,1.055609,21.087717,21.057574,20.853606
...,...,...,...,...,...,...,...,...
2021-03-02 15:43:55+00:00,47924.371719,1.893587,2.056611,1.391213,1.048328,28.196283,28.248115,28.052040
2021-03-02 15:43:56+00:00,47945.698929,1.894125,2.057068,1.390984,1.049013,28.190796,28.249486,28.042429
2021-03-02 15:43:57+00:00,47874.507523,1.893587,2.056840,1.385092,1.048100,28.199026,28.296884,28.041742


In [4]:
#sca = StandardScaler()
#index = data.index
#data = sca.fit_transform(data)
#data = pd.DataFrame(data, columns=cols, index=index)

data = data.diff().dropna()

In [ ]:
#X_train, X_test = train_test_split(data, test_size=0.4, shuffle = False,random_state = 42)

Extended Isolation Forest

In [5]:
# Calculate Silhouette Score. Not very successful.
from sklearn.metrics import silhouette_score

def get_score(ntrees, sample_size,X, threshold, ExtensionLevel=1):
    F1 = iso.iForest(np.array(X),ntrees=ntrees, sample_size=sample_size, ExtensionLevel=ExtensionLevel )
    pred=F1.compute_paths(np.array(X))
    pred_labels = np.where(pred>threshold,-1,1)
    score = silhouette_score(X, pred_labels)
    print("ntrees = ", ntrees, ", sample size= ", sample_size, ", score= ", score, ", threshold= ", threshold)

In [6]:
def scoring_function(model, df):
    pred=model.compute_paths(np.array(df))
    return pred*-1

In [ ]:
data

In [ ]:
'''for ntrees in [100, 150, 200]:
    for sample_size in [128, 256]:
        for threshold in [0.65, 0.68,0.7 ]:
            #get_score(ntree, size,X=X_train, threshold = threshold)
            #get_score(n_estimators, max_samples,contamination, X=X_train)
            model = iso.iForest(np.array(data),ntrees=ntrees, sample_size=sample_size, ExtensionLevel=1)
            #em, mv = emmv_scores(model, data, scoring_function).values()
            pred=model.compute_paths(np.array(data))
            pred_labels = np.where(pred>threshold,-1,1)
            ch_score = calinski_harabasz_score(data, pred)
            score = silhouette_score(data, pred_labels)
            print("ntrees = ", ntrees, ", sample size= ", sample_size, ", threshold= ", threshold)
            #print("em score: ", em, ", mv score: ", mv)
            print("silhouette score: ", score)
            print("ch_score = ", ch_score)
            print()
'''

In [ ]:
# Best hyperparameters : ntrees=100, sample_size=128, threshold=0.67

In [ ]:
for name, group in data.groupby(pd.Grouper(freq='10min')):
    F1 = iso.iForest(np.array(group),ntrees=200, sample_size=128, ExtensionLevel=1)
    pred=F1.compute_paths(np.array(group))

    em, mv = emmv_scores(F1, group, scoring_function).values()
    print("em: ", em, ", mv= ", mv)

    y_test_pred=np.where(pred>=0.6,-1,1)
    group['anomaly'] = y_test_pred
    print(name)
    print(group[group['anomaly']==-1])

In [ ]:
name = [i[0] for i in data.groupby(pd.Grouper(freq='10min'))]
groups = [i[1] for i in data.groupby(pd.Grouper(freq='10min'))]
group1 = groups[0]
anomalies = np.zeros(shape=(data.shape[0]))
pred = iso.iForest(np.array(group1),ntrees=200, sample_size=128, ExtensionLevel=1).compute_paths(np.array(group1))
anomalies[:len(group1+1)] = np.where(pred>=0.6,-1,1)

for n in range(1,len(groups)):
    group2 = groups[n]
    pred = iso.iForest(np.array(group1),ntrees=200, sample_size=128, ExtensionLevel=1).compute_paths(np.array(group2))
    anomalies[len(group1):len(group1)+len(group2)] = np.where(pred>=0.6,-1,1)
    group1 = pd.concat([group1, group2])

In [ ]:
print(data[anomalies==-1])

Score for the best hyperparameters on test set

In [7]:
F1 = iso.iForest(np.array(data),ntrees=200, sample_size=128, ExtensionLevel=1)
pred=F1.compute_paths(np.array(data))
pred = np.where(pred>=0.65,-1,1)

#em, mv = emmv_scores(F1, data, scoring_function).values()
#print("em: ", em, ", mv= ", mv)
print("silhouette score: ", silhouette_score(data, pred))
#print("ch_score= ", calinski_harabasz_score(data, pred))

silhouette score:  0.5845628449011667


In [8]:
final_data = data.copy()
final_data["Date"] = data.index
final_data["anomaly"] = pred

outliers=final_data.loc[final_data['anomaly']==-1]
outlier_index=list(outliers.index)


In [9]:
outliers1 = outliers.copy()
outliers1

name,Flow,P_1,P_3,P_4,P_5,T_2,T_3,T_4,Date,anomaly
Datetime,,,,,,,,,,
2021-03-02 10:11:38+00:00,-11.082458,-0.001077,-0.000686,0.006470,-0.000228,0.896090,0.555087,0.562866,2021-03-02 10:11:38+00:00,-1
2021-03-02 10:11:39+00:00,23.160553,0.001077,0.000686,-0.008566,-0.000228,-0.562730,-0.560570,-0.557374,2021-03-02 10:11:39+00:00,-1
2021-03-02 10:13:04+00:00,-109.585190,-0.075109,-0.000229,0.014588,0.000574,1.589977,1.435253,1.332962,2021-03-02 10:13:04+00:00,-1
2021-03-02 10:13:05+00:00,-32.886887,0.073764,-0.000915,-0.015274,-0.000487,-1.589291,-1.444849,-1.344549,2021-03-02 10:13:05+00:00,-1
2021-03-02 10:15:21+00:00,-140.975189,-0.033415,0.000229,-0.005372,-0.001136,0.360514,0.283130,0.233965,2021-03-02 10:15:21+00:00,-1
...,...,...,...,...,...,...,...,...,...,...
2021-03-02 15:39:28+00:00,-51.072693,-0.000538,-0.000229,-0.011411,-0.000228,-1.214283,-1.217934,-1.202726,2021-03-02 15:39:28+00:00,-1
2021-03-02 15:39:29+00:00,-10.921097,-0.000269,0.000457,0.004553,-0.000228,1.206053,1.217248,1.198012,2021-03-02 15:39:29+00:00,-1
2021-03-02 15:39:49+00:00,-82.071304,0.039012,-0.000229,0.001147,0.001655,-0.641206,-0.588059,-0.531905,2021-03-02 15:39:49+00:00,-1


In [10]:
 #Get the anomalies that appear consecutively, for instance three seconds in a row

outliers["date_delta"] = outliers["Date"] - outliers["Date"].shift()

keep = outliers.groupby("Date").date_delta.min() < pd.Timedelta(240,unit = "s")
outliers = outliers.set_index("Date")
outliers = outliers.loc[keep]
index = outliers
outliers = outliers.reset_index()

##Get only the number of anomalies that you want. For instance, take the first one when there are 3 anomalies in three consecutive seconds 
cond = outliers1["Date"].isin(outliers["Date"])
outliers1.drop(outliers1[cond].index,inplace = True)

outliers1.to_csv("anomaly")

outliers1

name,Flow,P_1,P_3,P_4,P_5,T_2,T_3,T_4,Date,anomaly
Datetime,,,,,,,,,,
2021-03-02 10:11:38+00:00,-11.082458,-0.001077,-0.000686,0.006470,-0.000228,0.896090,0.555087,0.562866,2021-03-02 10:11:38+00:00,-1
2021-03-02 10:22:54+00:00,89.088821,0.000269,-0.000457,0.008245,0.000214,0.935174,0.954950,0.691360,2021-03-02 10:22:54+00:00,-1
2021-03-02 10:39:45+00:00,67.916107,-0.043978,-0.032471,0.000686,-0.000850,2.411344,2.410611,0.009612,2021-03-02 10:39:45+00:00,-1
2021-03-02 10:47:42+00:00,120.849609,0.001077,-0.000229,-0.003929,-0.000177,-0.455209,-0.528665,-0.518161,2021-03-02 10:47:42+00:00,-1
2021-03-02 10:55:31+00:00,156.153488,-0.052800,-0.038953,-0.005339,-0.000228,2.501493,2.537155,2.543100,2021-03-02 10:55:31+00:00,-1
2021-03-02 10:59:48+00:00,-54.715347,-0.040667,-0.000457,0.009191,0.000413,1.244722,1.182944,1.108501,2021-03-02 10:59:48+00:00,-1
2021-03-02 11:12:14+00:00,-28.179932,-0.000269,-0.001743,-0.004431,-0.000280,-0.452468,-0.446568,-0.464704,2021-03-02 11:12:14+00:00,-1
2021-03-02 11:16:23+00:00,-103.999329,0.000807,0.000457,0.004794,0.000228,0.690280,0.669810,0.680309,2021-03-02 11:16:23+00:00,-1
2021-03-02 11:21:31+00:00,-904.751587,0.002276,-0.000229,0.000229,0.001375,-0.460701,-0.461978,0.006179,2021-03-02 11:21:31+00:00,-1


In [ ]:
data.reset_index(drop = True,inplace = True)

outliers=data[pred==-1]
outlier_index=list(outliers.index)

features = data.columns[0:8]
pca = PCA(n_components=3)
data_reduced = pca.fit_transform(data[features])

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = Axes3D(fig)

non_outlier = np.delete(data_reduced, outlier_index, 0)
ax.scatter(non_outlier[:,0], non_outlier[:,1], zs=non_outlier[:, 2], color = 'green')
ax.scatter(data_reduced[outlier_index,0],data_reduced[outlier_index,1], data_reduced[outlier_index,2],color='red')
plt.show()